Reference for this code:
https://www.geeksforgeeks.org/cosine-similarity/

In [ ]:
import random, string
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
spreadsheet = gc.open('results_llama2_3eps_500samples')
baseline_sheet = spreadsheet.worksheet('Baseline_200test')
llama2b_it_sheet = spreadsheet.worksheet('Llama2_it')

llama2bdata = llama2b_it_sheet.get_all_values()
llama2bdata = llama2bdata[1:201]

all_data_sheet = spreadsheet.worksheet('Clean_text_data')
all_data = all_data_sheet.get_all_values()
all_data = all_data[1:]

data = baseline_sheet.get_all_values()
data = data[1:201]


In [ ]:
sw = stopwords.words('english')
sw.append('.')
sw.append(',')
sw.append(':')
sw.append(';')
sw.append('\'')
sw.append('\"')

Getting cosines for baseline data

In [ ]:
import nltk
import math
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

texts = []

totalcosine = 0

cosine_sum_by_dataset = {}
cosine_sum_by_rewrite = {}
count_by_dataset = {}
count_by_rewrite = {}


for item in data:
  pred, actual, rewriteLabel, datasetLabel = item[:4]
  # print("pred", pred, '\n', "actual", actual, '\n', rewriteLabel, datasetLabel)

  #tokenization
  pred_list = word_tokenize(pred)
  actual_list = word_tokenize(actual)

  # print("pred_list", pred_list)
  # print("actual_list", actual_list)

  pred_list_final =[]; actual_list_final =[]

  #remove stop words from the string
  pred_set = {w for w in pred_list if not w in sw}
  actual_set = {w for w in actual_list if not w in sw}

  # print("pred_set", pred_set)
  # print("actual_set", actual_set)

  # form a set containing keywords of both strings
  rvector = pred_set.union(actual_set)
  for w in rvector:
    if w in pred_set: pred_list_final.append(1) # create a vector
    else: pred_list_final.append(0)
    if w in actual_set: actual_list_final.append(1)
    else: actual_list_final.append(0)

  #calculating cosine similarity
  c = 0

  for i in range(len(rvector)):
      c+= pred_list_final[i]*actual_list_final[i]

  cosine = c / float((sum(pred_list_final)*sum(actual_list_final))**0.5)
  # print("similarity: ", cosine)
  cosine_sum_by_dataset[datasetLabel] = cosine_sum_by_dataset.get(datasetLabel, 0)+cosine
  cosine_sum_by_rewrite[rewriteLabel] = cosine_sum_by_rewrite.get(rewriteLabel, 0)+cosine
  count_by_dataset[datasetLabel] = count_by_dataset.get(datasetLabel, 0)+1
  count_by_rewrite[rewriteLabel] = count_by_rewrite.get(rewriteLabel, 0)+1
  totalcosine += cosine

print("avg cosine value", totalcosine/len(data))
print("avg cosines for each dataset category")
for label in cosine_sum_by_dataset.keys():
  print("avg cosine for the label", label, "is", cosine_sum_by_dataset[label]/count_by_dataset[label])

for label in cosine_sum_by_rewrite.keys():
  print("avg cosine for the label", label, "is", cosine_sum_by_rewrite[label]/count_by_rewrite[label])



avg cosine value 0.41597306053804883
avg cosines for each dataset category
avg cosine for the label forum_messages is 0.4863005715182246
avg cosine for the label eli5-human is 0.5663790818948364
avg cosine for the label openwebtext is 0.2935773942312788
avg cosine for the label eli5-chatgpt is 0.40451514885908835
avg cosine for the label Poetry is 0.3333333333333333
avg cosine for the label Formality_fix is 0.4571976876381391
avg cosine for the label language is 0.18392908263381288
avg cosine for the label Theme_change is 0.655514963978329
avg cosine for the label Genre_change is 0.37222532307687933
avg cosine for the label Enhancement is 0.1178511301977579
avg cosine for the label music is 0.31843039275301616
avg cosine for the label creative is 0.7113005609164272
avg cosine for the label Perspective_shift is 0.2943261725754528
avg cosine for the label Active/Passive is 0.5955290742875216
avg cosine for the label Style_mimic is 0.3458968670179235
avg cosine for the label Paraphrasing 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Getting cosines for instruct llama model data


In [ ]:
import nltk
import math
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

texts = []
totalcosine = 0
cosine_sum_by_dataset = {}
cosine_sum_by_rewrite = {}
count_by_dataset = {}
count_by_rewrite = {}

for item in llama2bdata:
  pred, actual, rewriteLabel, datasetLabel = item[:4]
  # print("pred", pred, '\n', "actual", actual)

  #tokenization
  pred_list = word_tokenize(pred)
  actual_list = word_tokenize(actual)

  # print("pred_list", pred_list)
  # print("actual_list", actual_list)

  # sw contains the list of stopwords

  pred_list_final =[]; actual_list_final =[]

  #remove stop words from the string
  pred_set = {w for w in pred_list if not w in sw}
  actual_set = {w for w in actual_list if not w in sw}

  # print("pred_set", pred_set)
  # print("actual_set", actual_set)

  # form a set containing keywords of both strings
  rvector = pred_set.union(actual_set)
  for w in rvector:
    if w in pred_set: pred_list_final.append(1) # create a vector
    else: pred_list_final.append(0)
    if w in actual_set: actual_list_final.append(1)
    else: actual_list_final.append(0)

  # print(rvector)
  # print("predlistfinal", pred_list_final)
  # print("actuallistfinal", actual_list_final)

  #calculating cosine similarity
  c = 0

  for i in range(len(rvector)):
      c+= pred_list_final[i]*actual_list_final[i]

  cosine = c / float((sum(pred_list_final)*sum(actual_list_final))**0.5)
  # print("similarity: ", cosine)
  cosine_sum_by_dataset[datasetLabel] = cosine_sum_by_dataset.get(datasetLabel, 0)+cosine
  cosine_sum_by_rewrite[rewriteLabel] = cosine_sum_by_rewrite.get(rewriteLabel, 0)+cosine
  count_by_dataset[datasetLabel] = count_by_dataset.get(datasetLabel, 0)+1
  count_by_rewrite[rewriteLabel] = count_by_rewrite.get(rewriteLabel, 0)+1
  totalcosine += cosine

print("avg cosine value", totalcosine/len(llama2bdata))
print("avg cosines for each dataset category")
for label in cosine_sum_by_dataset.keys():
  print("avg cosine for the label", label, "is", cosine_sum_by_dataset[label]/count_by_dataset[label])
print("avg cosines for each prompt category")
for label in cosine_sum_by_rewrite.keys():
  print("avg cosine for the label", label, "is", cosine_sum_by_rewrite[label]/count_by_rewrite[label])


avg cosine value 0.021636266409769997
avg cosines for each dataset category
avg cosine for the label forum_messages is 0.036206819403675365
avg cosine for the label eli5-human is 0.014670122627921788
avg cosine for the label openwebtext is 0.02444025650251785
avg cosine for the label eli5-chatgpt is 0.008483186705082415
avg cosine for the label Poetry is 0.0
avg cosines for each prompt category
avg cosine for the label Formality_fix is 0.07494234806865929
avg cosine for the label language is 0.020239955147279835
avg cosine for the label Theme_change is 0.04151749878452431
avg cosine for the label Genre_change is 0.0
avg cosine for the label Enhancement is 0.0
avg cosine for the label music is 0.014344382763731171
avg cosine for the label creative is 0.008417937871268422
avg cosine for the label Perspective_shift is 0.0
avg cosine for the label Active/Passive is 0.020193929024003344
avg cosine for the label Style_mimic is 0.03863528444289931
avg cosine for the label Paraphrasing is 0.03

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
import math
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

totalcosine = 0
cosine_sum_by_dataset = {}
cosine_sum_by_rewrite = {}
count_by_dataset = {}
count_by_rewrite = {}

for item in all_data:
  pred, actual, rewriteLabel, datasetLabel = item[:4]
  # print("pred", pred, '\n', "actual", actual)

  #tokenization
  pred_list = word_tokenize(pred)
  actual_list = word_tokenize(actual)

  # print("pred_list", pred_list)
  # print("actual_list", actual_list)

  pred_list_final =[]; actual_list_final =[]

  #remove stop words from the string
  pred_set = {w for w in pred_list if not w in sw}
  actual_set = {w for w in actual_list if not w in sw}

  # form a set containing keywords of both strings
  rvector = pred_set.union(actual_set)
  for w in rvector:
    if w in pred_set: pred_list_final.append(1) # create a vector
    else: pred_list_final.append(0)
    if w in actual_set: actual_list_final.append(1)
    else: actual_list_final.append(0)

  #calculating cosine similarity
  c = 0

  for i in range(len(rvector)):
      c+= pred_list_final[i]*actual_list_final[i]

  cosine = c / float((sum(pred_list_final)*sum(actual_list_final))**0.5)
  # print("similarity: ", cosine)
  cosine_sum_by_dataset[datasetLabel] = cosine_sum_by_dataset.get(datasetLabel, 0)+cosine
  cosine_sum_by_rewrite[rewriteLabel] = cosine_sum_by_rewrite.get(rewriteLabel, 0)+cosine
  count_by_dataset[datasetLabel] = count_by_dataset.get(datasetLabel, 0)+1
  count_by_rewrite[rewriteLabel] = count_by_rewrite.get(rewriteLabel, 0)+1
  totalcosine += cosine

print("avg cosine value", totalcosine/len(all_data))
print("avg cosines for each dataset category")
for label in cosine_sum_by_dataset.keys():
  print("avg cosine for the label", label, "is", cosine_sum_by_dataset[label]/count_by_dataset[label])


print("****")
for label in cosine_sum_by_rewrite.keys():
  print("avg cosine for the label", label, "is", cosine_sum_by_rewrite[label]/count_by_rewrite[label])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


avg cosine value 0.35088633900929894
avg cosines for each dataset category
avg cosine for the label forum_messages is 0.3910106482745524
avg cosine for the label eli5-human is 0.539038921995597
avg cosine for the label openwebtext is 0.23847369714108924
avg cosine for the label eli5-chatgpt is 0.3702349499253311
avg cosine for the label Poetry is 0.33062511517097476
avg cosine for the label DatasetLabel is 0.0
****
avg cosine for the label Formality_fix is 0.31504552740194325
avg cosine for the label language is 0.29686894758688437
avg cosine for the label Theme_change is 0.46432370192888966
avg cosine for the label Genre_change is 0.33729460702581465
avg cosine for the label Enhancement is 0.15850797352677057
avg cosine for the label music is 0.2525528622574775
avg cosine for the label creative is 0.559466415997308
avg cosine for the label Perspective_shift is 0.2660930105908101
avg cosine for the label Active/Passive is 0.5338357265042409
avg cosine for the label Style_mimic is 0.224